In [1]:
from cameo import load_model
import cobra, pandas as pd
import matplotlib.pyplot as plt


#carregando o modelo
model = load_model('iYL1228_2_modificado.xml')


sol_biomassa = model.optimize()
biomassa_max = sol_biomassa.objective_value

print(sol_biomassa, biomassa_max)

<Solution 1.043 at 0x7f3d2a2480f0> 1.0426374979773874


In [2]:
def ajustar_meio(fonte_carbono, qtd_carbono, qtd_oxigenio, objetivo):
    global model    
    
    fonte_dict = {
        # glicose
        "glicose": "EX_glc__D_e","gli": "EX_glc__D_e", "glucose": "EX_glc__D_e",
        # glicerol
        "glicerol": "EX_glyc_e","gly": "EX_glyc_e", "glycerol": "EX_glyc_e", 
    }

    busca = fonte_carbono.lower() #transforma em letras minúsculas

    if busca in fonte_dict:
        fonte_id = fonte_dict[busca]
    else:
        try:
            fonte_id = model.reactions.get_by_id(fonte_carbono).id
        except KeyError:
            raise ValueError(f"Fonte de carbono '{fonte_carbono}' não encontrada no modelo.")


    c = model.reactions.get_by_id(fonte_id)
    c.lower_bound = -qtd_carbono
    c.upper_bound = qtd_carbono + 0.01


    # Ajuste do oxigênio
    if "EX_o2_e" in model.reactions:
        reac_o2 = model.reactions.get_by_id("EX_o2_e")
        reac_o2.lower_bound = -qtd_oxigenio
        reac_o2.upper_bound = qtd_oxigenio + 0.01
    else:
        raise ValueError("Reação 'EX_o2_e' não encontrada.")

    # Definir objetivo
    if objetivo in model.reactions:
        model.objective = objetivo
    else:
        raise ValueError(f"Reação '{objetivo}' não encontrada para função objetivo.")


In [3]:
import pandas as pd
import numpy as np

# Lista de reações que você quer monitorar
list_monitoramento = ["GAPD", "MDH", "13PPDH", "DCTR", "BTDD_RS", "BTDD_SS",
                      "ALCD2x", "LDH_D", "ALCD19", "GDH_R"]

# DataFrames globais
df_monitoramento_total = pd.DataFrame(columns=["Condicao", "Fracao", "Reacao", "Fluxo"])
df_top20_total = pd.DataFrame(columns=["Condicao", "Fracao", "Reacao", "Fluxo", "Rank_abs"])


def monitorar_fluxos(sol, frac, condicao):
    """
    Registra fluxos das reações monitoradas e o Top20 de maior fluxo absoluto.
    """
    global df_monitoramento_total, df_top20_total

    if sol is None:
        print("⚠️ monitorar_fluxos: solução vazia — pulando.")
        return

    # ----------------------------
    # 1) Registrar fluxos monitorados
    # ----------------------------
    rows = [{
        "Condicao": condicao,
        "Fracao": frac,
        "Reacao": rxn,
        "Fluxo": sol.fluxes.get(rxn, np.nan)
    } for rxn in list_monitoramento]

    df_monitoramento_total = pd.concat(
        [df_monitoramento_total, pd.DataFrame(rows)],
        ignore_index=True
    )

    # ----------------------------
    # 2) Top20 por módulo dos fluxos
    # ----------------------------
    try:
        fluxos_abs = sol.fluxes.abs().sort_values(ascending=False)
        top20_ids = fluxos_abs.head(20).index
    except Exception:
        top20_ids = []

    if len(top20_ids) > 0:
        rows_top = [{
            "Condicao": condicao,
            "Fracao": frac,
            "Reacao": rxn,
            "Fluxo": float(sol.fluxes.get(rxn, np.nan)),
            "Rank_abs": rank
        } for rank, rxn in enumerate(top20_ids, start=1)]

        df_top20_total = pd.concat(
            [df_top20_total, pd.DataFrame(rows_top)],
            ignore_index=True
        )

    # ----------------------------
    # 3) Log resumido
    # ----------------------------
    print(f"\n📊 Monitoramento — {condicao} | frac={frac:.2f}")
    print("\n  • Reações monitoradas:")
    print(pd.DataFrame(rows).to_string(index=False))

    if len(top20_ids) > 0:
        print("\n  • Top20 (abs) desta solução:")
        print(pd.DataFrame(rows_top).head(20).to_string(index=False))


In [4]:
import pandas as pd

# DataFrame global para armazenar os fluxos de fermentações
df_fermentacoes_total = pd.DataFrame()

def monitorar_fermentacoes(sol, condicao, frac, objetivo_alvo):

    global df_fermentacoes_total

    # Dicionário das fermentações a monitorar
    fermentacoes = {
        "Etanol": ("ALCD2x", "EX_etoh_e"),
        "L-Lactato": ("L_LACD2", "EX_lac__L_e"),
        "D-Lactato": ("LDH_D", "EX_lac__D_e"),
        "1,3-Propanodiol": ("13PPDH", "EX_13ppd_e"),
        "BDO meso": ("BTDD_MESO", "EX_btd__RS_e"),
        "BDO S,S": ("BTDD_SS", "EX_btd__SS_e"),
        "BDO R,R": ("GDH_R", "EX_btd_RR_e")
    }

    dados = []

    for nome, (reac_interna, reac_export) in fermentacoes.items():
        fluxo_interno = sol.fluxes.get(reac_interna, 0)
        fluxo_export = sol.fluxes.get(reac_export, 0)

        dados.append({
            "Condição": condicao,
            "Fração Biomassa": frac,
            "Fermentação": nome,
            "Reação interna": reac_interna,
            "Fluxo interno (mmol/gDW/h)": fluxo_interno,
            "Exportação": reac_export,
            "Fluxo exportado (mmol/gDW/h)": fluxo_export,
            "Alvo da simulação": objetivo_alvo
        })

    df_local = pd.DataFrame(dados)
    df_fermentacoes_total = pd.concat([df_fermentacoes_total, df_local], ignore_index=True)


In [5]:
import pandas as pd

# DataFrame global onde vamos guardar todos os resultados
resultados_totais = pd.DataFrame(columns=["Condicao", "Fracao", "Status", "Biomassa", "Fluxo_alvo"])

def registrar_resultado(condicao, fracao, status, biomassa, alvo):
    """Registra cada resultado em um DataFrame global"""
    global resultados_totais
    
    resultados_totais = pd.concat([
        resultados_totais,
        pd.DataFrame([{
            "Condicao": condicao,
            "Fracao": fracao,
            "Status": status,
            "Biomassa": biomassa,
            "Fluxo_alvo": alvo
        }])
    ], ignore_index=True)



In [6]:
def salvar_resultados(nome_arquivo="resumo_simulacoes.xlsx"):
    global resultados_totais
    resultados_totais.to_excel(nome_arquivo, index=False)
    print(f"Resultados consolidados salvos em '{nome_arquivo}'")

In [7]:
def aplicar_nocautes(model, lista_reacoes):
    """
    Aplica knockouts a uma lista de reações e retorna quais foram efetivamente aplicados.
    """
    knockouts_aplicados = []

    for rxn_id in lista_reacoes:
        try:
            rxn = model.reactions.get_by_id(rxn_id)
            rxn.knock_out()  # método nativo do cobrapy
            print(f"✅ Knockout aplicado: {rxn_id}")
            knockouts_aplicados.append(rxn_id)
        except KeyError:
            print(f"⚠️ Reação '{rxn_id}' não encontrada no modelo. Ignorada.")
    
    return knockouts_aplicados


In [8]:
# 🔹 DataFrame global (pode ser alimentado pelas simulações)
df_fluxos_fermentacoes = pd.DataFrame(columns=["Condicao", "Fracao", "Reacao", "Fluxo"])


# ======================================================
# 📉 Função para plotar os fluxos das fermentações
# ======================================================

def plotar_fluxos_fermentacoes(df_fluxos_fermentacoes, condicao):
    """
    Gera gráfico de fluxos de fermentação para uma condição específica,
    mostrando apenas as reações de exchange (EX_).
    """
    import matplotlib.pyplot as plt

    subset = df_fluxos_fermentacoes[df_fluxos_fermentacoes["Condicao"] == condicao]

    # 🔹 Filtra apenas as reações de exchange
    subset = subset[subset["Reacao"].str.contains("EX_", case=False, na=False)]

    if subset.empty:
        print("⚠️ Nenhuma reação de exchange encontrada para plotar.")
        return

    plt.figure(figsize=(10, 6))
    for reacao in subset["Reacao"].unique():
        dados = subset[subset["Reacao"] == reacao]
        plt.plot(dados["Fracao"], dados["Fluxo"], marker='o', label=reacao)

    plt.title(f"Fluxos de reações de exchange — {condicao}")
    plt.xlabel("Fração da biomassa")
    plt.ylabel("Fluxo (mmol/gDW/h)")
    plt.legend()
    plt.grid(True)
    plt.show()


# ======================================================
# 🧩 Função para aplicar knockout dinâmico
# ======================================================
def aplicar_knockout_dinamico(model, df_fluxos_fermentacoes, condicao, threshold=1e-4):
    subset = df_fluxos_fermentacoes[
        (df_fluxos_fermentacoes["Condicao"] == condicao)
        & (df_fluxos_fermentacoes["Fluxo"].abs() > threshold)
    ]

    reacoes_para_nocautear = subset["Reacao"].unique()

    if len(reacoes_para_nocautear) == 0:
        print("Nenhuma reação significativa para nocaute.")
        return []

    print("\n🧬 Reações candidatas a knockout dinâmico:")
    for rxn in reacoes_para_nocautear:
        print(f" - {rxn}")
        try:
            model.reactions.get_by_id(rxn).knock_out()
        except KeyError:
            print(f"   ⚠️ Reação {rxn} não encontrada no modelo.")
    
    return list(reacoes_para_nocautear)


In [10]:

def simular_fba_iterativo(model, fonte_carbono, qtd_carbono, qtd_oxigenio,
                          objetivo_biomassa, objetivos_alvo, condicao,
                          passos=10, nocautes=None, aplicar_knockout_dinamico_flag=False):
    """
    Executa uma simulação FBA iterativa, registra fluxos de fermentações,
    plota resultados e pode aplicar knockout dinâmico opcional.
    """

    global df_fluxos_fermentacoes
    
    with model:
        print(f"\n=== Iniciando simulação: {condicao} ===")
        print("Ajustando meio e otimizando biomassa máxima...\n")

        ajustar_meio(fonte_carbono, qtd_carbono, qtd_oxigenio, objetivo_biomassa)
        sol_biomax = model.optimize()

        if sol_biomax.status != "optimal":
            raise RuntimeError(f"Não foi possível otimizar a biomassa em {condicao}.")

        biomassa_max = sol_biomax.objective_value
        print(f"Biomassa máxima ({condicao}): {biomassa_max:.4f}\n")

        
         # Lista que armazenará knockouts dinâmicos
        knockouts_dinamicos = set()
        
        fracoes_biomassa = [0.00] + [i/10 for i in range(1, 11)]

        for frac in fracoes_biomassa:
            biomassa_min = frac * biomassa_max

       
            

            with model:
                knockouts_aplicados = []

                # 🔹 Knockout automático 1: GDH_R (exceto glicerol)
                if not any(s in fonte_carbono.lower() for s in ["glycerol", "glyc", "glicerol"]):
                    try:
                        model.reactions.get_by_id("GDH_R").knock_out()
                        knockouts_aplicados.append("GDH_R")
                    except KeyError:
                        pass

                # 🔹 Knockout automático 2: DAC (anaeróbico)
                if qtd_oxigenio <= 0:
                    try:
                        model.reactions.get_by_id("DAC").knock_out()
                        knockouts_aplicados.append("DAC")
                    except KeyError:
                        pass

                # 🔹 Knockouts manuais
                if nocautes:
                    for k in nocautes:
                        try:
                            model.reactions.get_by_id(k).knock_out()
                            knockouts_aplicados.append(k)
                        except KeyError:
                            print(f"⚠️ Reação {k} não encontrada.")
                 
                # 🔹 Knockouts dinâmicos acumulados
                for rxn in knockouts_dinamicos:
                    try:
                        model.reactions.get_by_id(rxn).knock_out()
                        knockouts_aplicados.append(rxn)
                    except:
                        pass
                            
                print(f"Knockouts aplicados: {', '.join(knockouts_aplicados) or 'nenhum'}")

                # 🔹 Restringe biomassa
                reac_biomassa = model.reactions.get_by_id(objetivo_biomassa)
                reac_biomassa.lower_bound = biomassa_min
                
                
                # 🔹 Define função objetivo (única ou múltipla)
                if isinstance(objetivos_alvo, list):
                    expr = sum(model.reactions.get_by_id(obj).flux_expression for obj in objetivos_alvo)
                    model.objective = expr
                    print(f"Função objetivo composta: {', '.join(objetivos_alvo)}")
                else:
                    model.objective = objetivos_alvo

                sol = model.optimize()
                print(f"\n=== Fração {frac:.1f} da biomassa ===\nStatus: {sol.status}")

                if sol.status != "optimal":
                    print("⚠️ Solução inviável nesta fração.\n")
               
            
                # 🔹 Exibir biomassa e valor da função objetivo
                biomassa_fluxo = sol.fluxes.get(objetivo_biomassa, 0)
                objetivo_valor = sol.objective_value

                # chama monitoramento
                monitorar_fluxos(sol=sol, frac=frac, condicao=condicao)

                
                print(f"Biomassa atingida: {biomassa_fluxo:.4f}")
                print(f"Valor da função objetivo: {objetivo_valor:.4f}")
                
       
                registrar_resultado(condicao=condicao, fracao=frac,status=sol.status,biomassa=biomassa_fluxo, alvo=objetivo_valor)

                
                
                # -------------------------------------------------------------
                # 🔥 PARTE IMPORTANTE: knockout dinâmico das reações de objetivo
                # -------------------------------------------------------------
                if isinstance(objetivos_alvo, list):
                    print("\n🔍 Verificando reações do objetivo para knockout dinâmico:")

                    for obj in objetivos_alvo:
                        fluxo_obj = sol.fluxes.get(obj, 0)

                        print(f"  {obj}: fluxo = {fluxo_obj:.4f}")

                        # Se teve fluxo → adicionar aos knockouts do próximo ciclo
                        if abs(fluxo_obj) > 1e-6:
                            knockouts_dinamicos.add(obj)
                            print(f"    → Será nocauteada na próxima iteração!")
            
            
                
                # 🔹 Se o objetivo for etanol, monitorar fermentações
                if any("etoh" in obj.lower() for obj in (objetivos_alvo if isinstance(objetivos_alvo, list) else [objetivos_alvo])):
                    fermentacoes = {
                        "Etanol": ("ALCD2x", "EX_etoh_e"),
                        "L-Lactato": ("L_LACD2", "EX_lac__L_e"),
                        "D-Lactato": ("LDH_D", "EX_lac__D_e"),
                        "1,3-Propanodiol": ("13PPDH", "EX_13ppd_e"),
                        "BDO meso": ("BTDD_MESO", "EX_btd__RS_e"),
                        "BDO S,S": ("BTDD_SS", "EX_btd__SS_e"),
                    }

                    registros = []
                    print("\n🔍 Monitorando fermentações alternativas:")
                    for nome, (rxn_interna, rxn_excrecao) in fermentacoes.items():
                        fluxo_interno = sol.fluxes.get(rxn_interna, 0)
                        fluxo_excrecao = sol.fluxes.get(rxn_excrecao, 0)
                        print(f"  {nome}: {rxn_interna}={fluxo_interno:.4f}, {rxn_excrecao}={fluxo_excrecao:.4f}")

                        registros.append({
                            "Condicao": condicao,
                            "Fracao": frac,
                            "Reacao": rxn_excrecao,
                            "Fluxo": fluxo_excrecao
                        })
                        registros.append({
                            "Condicao": condicao,
                            "Fracao": frac,
                            "Reacao": rxn_interna,
                            "Fluxo": fluxo_interno
                        })
       
        # 🔹 Ao final: gerar gráfico e aplicar knockout dinâmico (opcional)
        if (not df_fluxos_fermentacoes.empty 
            and "etanol" in condicao.lower() 
            and isinstance(objetivos_alvo, (list, tuple)) 
            and len(objetivos_alvo) > 1):

            print("\n📊 Gerando gráfico de fermentações (apenas para múltiplos objetivos de etanol)...")
            plotar_fluxos_fermentacoes(df_fluxos_fermentacoes, condicao)
            
            if aplicar_knockout_dinamico_flag:
                aplicar_knockout_dinamico(model, df_fluxos_fermentacoes, condicao)
        return resultados_totais
        print(f"\n✅ Simulação '{condicao}' concluída!\n")


In [11]:
#Glicose e etanol 
# Aeróbico
#monitorar outras fermentações
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_etoh_e", 
                      condicao="Aeróbico etanol")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_etoh_e", 
                      condicao="Anaeróbico etanol")





=== Iniciando simulação: Aeróbico etanol ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico etanol): 1.8652

Knockouts aplicados: GDH_R

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol | frac=0.00

  • Reações monitoradas:
        Condicao  Fracao   Reacao  Fluxo
 Aeróbico etanol     0.0     GAPD  37.00
 Aeróbico etanol     0.0      MDH  13.15
 Aeróbico etanol     0.0   13PPDH   0.00
 Aeróbico etanol     0.0     DCTR   0.00
 Aeróbico etanol     0.0  BTDD_RS  31.65
 Aeróbico etanol     0.0  BTDD_SS   0.00
 Aeróbico etanol     0.0   ALCD2x -37.00
 Aeróbico etanol     0.0    LDH_D   0.00
 Aeróbico etanol     0.0   ALCD19   0.00
 Aeróbico etanol     0.0    GDH_R   0.00

  • Top20 (abs) desta solução:
        Condicao  Fracao     Reacao  Fluxo  Rank_abs
 Aeróbico etanol     0.0     CO2tpp -37.00         1
 Aeróbico etanol     0.0   EX_co2_e  37.00         2
 Aeróbico etanol     0.0     CO2tex -37.00         3
 Aeróbico etanol    

Knockouts aplicados: GDH_R

=== Fração 0.4 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol | frac=0.40

  • Reações monitoradas:
        Condicao  Fracao   Reacao      Fluxo
 Aeróbico etanol     0.4     GAPD  34.968052
 Aeróbico etanol     0.4      MDH   5.267793
 Aeróbico etanol     0.4   13PPDH   0.000000
 Aeróbico etanol     0.4     DCTR   0.000000
 Aeróbico etanol     0.4  BTDD_RS  11.722636
 Aeróbico etanol     0.4  BTDD_SS   0.000000
 Aeróbico etanol     0.4   ALCD2x -24.214468
 Aeróbico etanol     0.4    LDH_D   0.000000
 Aeróbico etanol     0.4   ALCD19   0.000000
 Aeróbico etanol     0.4    GDH_R   0.000000

  • Top20 (abs) desta solução:
        Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Aeróbico etanol     0.4      ATPS4rpp  37.683345         1
 Aeróbico etanol     0.4           PGK -34.968052         2
 Aeróbico etanol     0.4          GAPD  34.968052         3
 Aeróbico etanol     0.4        CO2tpp -34.130676         4
 Aeróbico etanol     0.4 


=== Fração 1.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol | frac=1.00

  • Reações monitoradas:
        Condicao  Fracao   Reacao         Fluxo
 Aeróbico etanol     1.0     GAPD  3.192013e+01
 Aeróbico etanol     1.0      MDH  1.820565e+01
 Aeróbico etanol     1.0   13PPDH  0.000000e+00
 Aeróbico etanol     1.0     DCTR  0.000000e+00
 Aeróbico etanol     1.0  BTDD_RS  3.125551e+00
 Aeróbico etanol     1.0  BTDD_SS  0.000000e+00
 Aeróbico etanol     1.0   ALCD2x  2.477121e-13
 Aeróbico etanol     1.0    LDH_D  0.000000e+00
 Aeróbico etanol     1.0   ALCD19  0.000000e+00
 Aeróbico etanol     1.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
        Condicao  Fracao        Reacao       Fluxo  Rank_abs
 Aeróbico etanol     1.0      ATPS4rpp  134.472192         1
 Aeróbico etanol     1.0        H2Otpp  -85.092603         2
 Aeróbico etanol     1.0        H2Otex  -85.092603         3
 Aeróbico etanol     1.0      EX_h2o_e   85.092603         4
 Aeróbico etan


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico etanol | frac=0.50

  • Reações monitoradas:
          Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol     0.5     GAPD  36.314715
 Anaeróbico etanol     0.5      MDH   0.190974
 Anaeróbico etanol     0.5   13PPDH   0.000000
 Anaeróbico etanol     0.5     DCTR   0.000000
 Anaeróbico etanol     0.5  BTDD_RS  18.132911
 Anaeróbico etanol     0.5  BTDD_SS   0.000000
 Anaeróbico etanol     0.5   ALCD2x -33.709057
 Anaeróbico etanol     0.5    LDH_D   0.000000
 Anaeróbico etanol     0.5   ALCD19   0.000000
 Anaeróbico etanol     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
          Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico etanol     0.5          GAPD  36.314715         1
 Anaeróbico etanol     0.5           PGK -36.314715         2
 Anaeróbico etanol     0.5           PGM -36.090826         3
 Anaeróbico etanol     0.5           ENO  36.090826         4
 Anaeróbico etanol


=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico etanol | frac=0.90

  • Reações monitoradas:
          Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol     0.9     GAPD  35.766487
 Anaeróbico etanol     0.9      MDH   0.343754
 Anaeróbico etanol     0.9   13PPDH   0.000000
 Anaeróbico etanol     0.9     DCTR   0.000000
 Anaeróbico etanol     0.9  BTDD_RS  17.692232
 Anaeróbico etanol     0.9  BTDD_SS   0.000000
 Anaeróbico etanol     0.9   ALCD2x -22.570460
 Anaeróbico etanol     0.9    LDH_D   0.000000
 Anaeróbico etanol     0.9   ALCD19   0.000000
 Anaeróbico etanol     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
          Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico etanol     0.9          GAPD  35.766487         1
 Anaeróbico etanol     0.9           PGK -35.766487         2
 Anaeróbico etanol     0.9           PGM -35.069469         3
 Anaeróbico etanol     0.9           ENO  35.069469         4
 Anaeróbico etanol

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,3.700000e+01
1,Aeróbico etanol,0.1,optimal,0.186515,3.456049e+01
2,Aeróbico etanol,0.2,optimal,0.373031,3.212099e+01
3,Aeróbico etanol,0.3,optimal,0.559546,2.825021e+01
4,Aeróbico etanol,0.4,optimal,0.746062,2.421447e+01
5,Aeróbico etanol,0.5,optimal,0.932577,2.017872e+01
6,Aeróbico etanol,0.6,optimal,1.119092,1.614298e+01
7,Aeróbico etanol,0.7,optimal,1.305608,1.210723e+01
8,Aeróbico etanol,0.8,optimal,1.492123,8.071489e+00
9,Aeróbico etanol,0.9,optimal,1.678638,4.035745e+00


In [12]:
#Glicose e R,R-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd_RR_e", 
                      condicao="Aeróbico R,R-BDO")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd_RR_e", 
                      condicao="Anaeróbico R,R-BDO")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: Aeróbico R,R-BDO ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico R,R-BDO): 1.8652

Knockouts aplicados: GDH_R

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico R,R-BDO | frac=0.00

  • Reações monitoradas:
         Condicao  Fracao   Reacao      Fluxo
 Aeróbico R,R-BDO     0.0     GAPD  31.920129
 Aeróbico R,R-BDO     0.0      MDH  14.021068
 Aeróbico R,R-BDO     0.0   13PPDH   0.000000
 Aeróbico R,R-BDO     0.0     DCTR   0.000000
 Aeróbico R,R-BDO     0.0  BTDD_RS   0.000000
 Aeróbico R,R-BDO     0.0  BTDD_SS   0.000000
 Aeróbico R,R-BDO     0.0   ALCD2x   0.000000
 Aeróbico R,R-BDO     0.0    LDH_D   0.000000
 Aeróbico R,R-BDO     0.0   ALCD19   0.000000
 Aeróbico R,R-BDO     0.0    GDH_R   0.000000

  • Top20 (abs) desta solução:
         Condicao  Fracao        Reacao       Fluxo  Rank_abs
 Aeróbico R,R-BDO     0.0      ATPS4rpp  134.472192         1
 Aeróbico R,R-BDO     0.0        H2Otpp  -85.09260


📊 Monitoramento — Aeróbico R,R-BDO | frac=0.60

  • Reações monitoradas:
         Condicao  Fracao   Reacao      Fluxo
 Aeróbico R,R-BDO     0.6     GAPD  31.920129
 Aeróbico R,R-BDO     0.6      MDH  14.021068
 Aeróbico R,R-BDO     0.6   13PPDH   0.000000
 Aeróbico R,R-BDO     0.6     DCTR   0.000000
 Aeróbico R,R-BDO     0.6  BTDD_RS   0.000000
 Aeróbico R,R-BDO     0.6  BTDD_SS   0.000000
 Aeróbico R,R-BDO     0.6   ALCD2x   0.000000
 Aeróbico R,R-BDO     0.6    LDH_D   0.000000
 Aeróbico R,R-BDO     0.6   ALCD19   0.000000
 Aeróbico R,R-BDO     0.6    GDH_R   0.000000

  • Top20 (abs) desta solução:
         Condicao  Fracao       Reacao       Fluxo  Rank_abs
 Aeróbico R,R-BDO     0.6     ATPS4rpp  134.472192         1
 Aeróbico R,R-BDO     0.6       H2Otex  -85.092603         2
 Aeróbico R,R-BDO     0.6       H2Otpp  -85.092603         3
 Aeróbico R,R-BDO     0.6     EX_h2o_e   85.092603         4
 Aeróbico R,R-BDO     0.6   CYTBO3_4pp   75.629630         5
 Aeróbico R,R-BDO     


📊 Monitoramento — Anaeróbico R,R-BDO | frac=0.20

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 Anaeróbico R,R-BDO     0.2     GAPD  35.629430
 Anaeróbico R,R-BDO     0.2      MDH   0.381949
 Anaeróbico R,R-BDO     0.2   13PPDH   0.000000
 Anaeróbico R,R-BDO     0.2     DCTR   0.000000
 Anaeróbico R,R-BDO     0.2  BTDD_RS  17.602480
 Anaeróbico R,R-BDO     0.2  BTDD_SS   0.000000
 Anaeróbico R,R-BDO     0.2   ALCD2x -15.122733
 Anaeróbico R,R-BDO     0.2    LDH_D   0.000000
 Anaeróbico R,R-BDO     0.2   ALCD19   0.000000
 Anaeróbico R,R-BDO     0.2    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico R,R-BDO     0.2           PGK -35.629430         1
 Anaeróbico R,R-BDO     0.2          GAPD  35.629430         2
 Anaeróbico R,R-BDO     0.2           ENO  34.854966         3
 Anaeróbico R,R-BDO     0.2           PGM -34.854966         4
 Anaeróbico R,R-BDO     0.2           PFL  31.843


=== Fração 0.6 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico R,R-BDO | frac=0.60

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 Anaeróbico R,R-BDO     0.6     GAPD  35.629430
 Anaeróbico R,R-BDO     0.6      MDH   0.381949
 Anaeróbico R,R-BDO     0.6   13PPDH   0.000000
 Anaeróbico R,R-BDO     0.6     DCTR   0.000000
 Anaeróbico R,R-BDO     0.6  BTDD_RS  17.602480
 Anaeróbico R,R-BDO     0.6  BTDD_SS   0.000000
 Anaeróbico R,R-BDO     0.6   ALCD2x -15.122733
 Anaeróbico R,R-BDO     0.6    LDH_D   0.000000
 Anaeróbico R,R-BDO     0.6   ALCD19   0.000000
 Anaeróbico R,R-BDO     0.6    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico R,R-BDO     0.6          GAPD  35.629430         1
 Anaeróbico R,R-BDO     0.6           PGK -35.629430         2
 Anaeróbico R,R-BDO     0.6           ENO  34.854966         3
 Anaeróbico R,R-BDO     0.6           PGM -34.854966         4
 

Knockouts aplicados: GDH_R, DAC

=== Fração 1.0 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico R,R-BDO | frac=1.00

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 Anaeróbico R,R-BDO     1.0     GAPD  35.629430
 Anaeróbico R,R-BDO     1.0      MDH   0.381949
 Anaeróbico R,R-BDO     1.0   13PPDH   0.000000
 Anaeróbico R,R-BDO     1.0     DCTR   0.000000
 Anaeróbico R,R-BDO     1.0  BTDD_RS  17.602480
 Anaeróbico R,R-BDO     1.0  BTDD_SS   0.000000
 Anaeróbico R,R-BDO     1.0   ALCD2x -15.122733
 Anaeróbico R,R-BDO     1.0    LDH_D   0.000000
 Anaeróbico R,R-BDO     1.0   ALCD19   0.000000
 Anaeróbico R,R-BDO     1.0    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico R,R-BDO     1.0          GAPD  35.629430         1
 Anaeróbico R,R-BDO     1.0           PGK -35.629430         2
 Anaeróbico R,R-BDO     1.0           ENO  34.854966         3
 Anaeróbico R,R-BDO     1.0     

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,3.700000e+01
1,Aeróbico etanol,0.1,optimal,0.186515,3.456049e+01
2,Aeróbico etanol,0.2,optimal,0.373031,3.212099e+01
3,Aeróbico etanol,0.3,optimal,0.559546,2.825021e+01
4,Aeróbico etanol,0.4,optimal,0.746062,2.421447e+01
5,Aeróbico etanol,0.5,optimal,0.932577,2.017872e+01
6,Aeróbico etanol,0.6,optimal,1.119092,1.614298e+01
7,Aeróbico etanol,0.7,optimal,1.305608,1.210723e+01
8,Aeróbico etanol,0.8,optimal,1.492123,8.071489e+00
9,Aeróbico etanol,0.9,optimal,1.678638,4.035745e+00


In [13]:
#Glicose e S,S-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=20, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__SS_e", 
                      condicao="Aeróbico S,S-BDO")




# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__SS_e", 
                      condicao="Anaeróbico S,S-BDO")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: Aeróbico S,S-BDO ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico S,S-BDO): 1.3261

Knockouts aplicados: GDH_R

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico S,S-BDO | frac=0.00

  • Reações monitoradas:
         Condicao  Fracao   Reacao         Fluxo
 Aeróbico S,S-BDO     0.0     GAPD  3.700000e+01
 Aeróbico S,S-BDO     0.0      MDH -7.249372e-14
 Aeróbico S,S-BDO     0.0   13PPDH  0.000000e+00
 Aeróbico S,S-BDO     0.0     DCTR  1.850000e+01
 Aeróbico S,S-BDO     0.0  BTDD_RS  1.850000e+01
 Aeróbico S,S-BDO     0.0  BTDD_SS  0.000000e+00
 Aeróbico S,S-BDO     0.0   ALCD2x  0.000000e+00
 Aeróbico S,S-BDO     0.0    LDH_D  1.137286e-13
 Aeróbico S,S-BDO     0.0   ALCD19  0.000000e+00
 Aeróbico S,S-BDO     0.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
         Condicao  Fracao        Reacao  Fluxo  Rank_abs
 Aeróbico S,S-BDO     0.0           ENO   37.0         1
 Aeróbico S,S-BDO     0.0  


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico S,S-BDO | frac=0.50

  • Reações monitoradas:
         Condicao  Fracao   Reacao      Fluxo
 Aeróbico S,S-BDO     0.5     GAPD  35.194099
 Aeróbico S,S-BDO     0.5      MDH   5.112707
 Aeróbico S,S-BDO     0.5   13PPDH   0.000000
 Aeróbico S,S-BDO     0.5     DCTR  12.602939
 Aeróbico S,S-BDO     0.5  BTDD_RS  15.073504
 Aeróbico S,S-BDO     0.5  BTDD_SS   0.000000
 Aeróbico S,S-BDO     0.5   ALCD2x   0.000000
 Aeróbico S,S-BDO     0.5    LDH_D   0.000000
 Aeróbico S,S-BDO     0.5   ALCD19   0.000000
 Aeróbico S,S-BDO     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
         Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Aeróbico S,S-BDO     0.5           PGK -35.194099         1
 Aeróbico S,S-BDO     0.5          GAPD  35.194099         2
 Aeróbico S,S-BDO     0.5           ENO  34.173643         3
 Aeróbico S,S-BDO     0.5           PGM -34.173643         4
 Aeróbico S,S-BDO     0.5      ATPS


📊 Monitoramento — Anaeróbico S,S-BDO | frac=0.00

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 Anaeróbico S,S-BDO     0.0     GAPD  35.629430
 Anaeróbico S,S-BDO     0.0      MDH   0.381949
 Anaeróbico S,S-BDO     0.0   13PPDH   0.000000
 Anaeróbico S,S-BDO     0.0     DCTR   0.000000
 Anaeróbico S,S-BDO     0.0  BTDD_RS   2.479747
 Anaeróbico S,S-BDO     0.0  BTDD_SS   0.000000
 Anaeróbico S,S-BDO     0.0   ALCD2x -15.122733
 Anaeróbico S,S-BDO     0.0    LDH_D   0.000000
 Anaeróbico S,S-BDO     0.0   ALCD19   0.000000
 Anaeróbico S,S-BDO     0.0    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico S,S-BDO     0.0          GAPD  35.629430         1
 Anaeróbico S,S-BDO     0.0           PGK -35.629430         2
 Anaeróbico S,S-BDO     0.0           PGM -34.854966         3
 Anaeróbico S,S-BDO     0.0           ENO  34.854966         4
 Anaeróbico S,S-BDO     0.0           PFL  31.843


=== Fração 0.7 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico S,S-BDO | frac=0.70

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 Anaeróbico S,S-BDO     0.7     GAPD  35.629430
 Anaeróbico S,S-BDO     0.7      MDH   0.381949
 Anaeróbico S,S-BDO     0.7   13PPDH   0.000000
 Anaeróbico S,S-BDO     0.7     DCTR   0.000000
 Anaeróbico S,S-BDO     0.7  BTDD_RS   2.479747
 Anaeróbico S,S-BDO     0.7  BTDD_SS   0.000000
 Anaeróbico S,S-BDO     0.7   ALCD2x -15.122733
 Anaeróbico S,S-BDO     0.7    LDH_D   0.000000
 Anaeróbico S,S-BDO     0.7   ALCD19   0.000000
 Anaeróbico S,S-BDO     0.7    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico S,S-BDO     0.7           PGK -35.629430         1
 Anaeróbico S,S-BDO     0.7          GAPD  35.629430         2
 Anaeróbico S,S-BDO     0.7           ENO  34.854966         3
 Anaeróbico S,S-BDO     0.7           PGM -34.854966         4
 

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,37.000000
1,Aeróbico etanol,0.1,optimal,0.186515,34.560494
2,Aeróbico etanol,0.2,optimal,0.373031,32.120988
3,Aeróbico etanol,0.3,optimal,0.559546,28.250213
4,Aeróbico etanol,0.4,optimal,0.746062,24.214468
...,...,...,...,...,...
61,"Anaeróbico S,S-BDO",0.6,optimal,0.503226,0.000000
62,"Anaeróbico S,S-BDO",0.7,optimal,0.503226,0.000000
63,"Anaeróbico S,S-BDO",0.8,optimal,0.503226,0.000000
64,"Anaeróbico S,S-BDO",0.9,optimal,0.503226,0.000000


In [14]:
#Glicose e meso-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__RS_e", 
                      condicao="Aeróbico meso-BDO")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__RS_e", 
                      condicao="Anaeróbico meso-BDO")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: Aeróbico meso-BDO ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico meso-BDO): 1.8652

Knockouts aplicados: GDH_R

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico meso-BDO | frac=0.00

  • Reações monitoradas:
          Condicao  Fracao   Reacao         Fluxo
 Aeróbico meso-BDO     0.0     GAPD  3.700000e+01
 Aeróbico meso-BDO     0.0      MDH -1.482298e-14
 Aeróbico meso-BDO     0.0   13PPDH  0.000000e+00
 Aeróbico meso-BDO     0.0     DCTR  0.000000e+00
 Aeróbico meso-BDO     0.0  BTDD_RS  1.850000e+01
 Aeróbico meso-BDO     0.0  BTDD_SS  0.000000e+00
 Aeróbico meso-BDO     0.0   ALCD2x  0.000000e+00
 Aeróbico meso-BDO     0.0    LDH_D  0.000000e+00
 Aeróbico meso-BDO     0.0   ALCD19  0.000000e+00
 Aeróbico meso-BDO     0.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
          Condicao  Fracao        Reacao  Fluxo  Rank_abs
 Aeróbico meso-BDO     0.0          NTP1   67.2         1
 Aeróbico m


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico meso-BDO | frac=0.50

  • Reações monitoradas:
          Condicao  Fracao   Reacao      Fluxo
 Aeróbico meso-BDO     0.5     GAPD  34.460065
 Aeróbico meso-BDO     0.5      MDH   5.992847
 Aeróbico meso-BDO     0.5   13PPDH   0.000000
 Aeróbico meso-BDO     0.5     DCTR   0.000000
 Aeróbico meso-BDO     0.5  BTDD_RS  15.339159
 Aeróbico meso-BDO     0.5  BTDD_SS   0.000000
 Aeróbico meso-BDO     0.5   ALCD2x   0.000000
 Aeróbico meso-BDO     0.5    LDH_D   0.000000
 Aeróbico meso-BDO     0.5   ALCD19   0.000000
 Aeróbico meso-BDO     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
          Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Aeróbico meso-BDO     0.5      ATPS4rpp  55.190641         1
 Aeróbico meso-BDO     0.5        H2Otpp -44.159938         2
 Aeróbico meso-BDO     0.5        H2Otex -44.159938         3
 Aeróbico meso-BDO     0.5      EX_h2o_e  44.159938         4
 Aeróbico meso-BDO


=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico meso-BDO | frac=0.90

  • Reações monitoradas:
          Condicao  Fracao   Reacao      Fluxo
 Aeróbico meso-BDO     0.9     GAPD  32.428116
 Aeróbico meso-BDO     0.9      MDH  15.774179
 Aeróbico meso-BDO     0.9   13PPDH   0.000000
 Aeróbico meso-BDO     0.9     DCTR   0.000000
 Aeróbico meso-BDO     0.9  BTDD_RS   5.579360
 Aeróbico meso-BDO     0.9  BTDD_SS   0.000000
 Aeróbico meso-BDO     0.9   ALCD2x   0.000000
 Aeróbico meso-BDO     0.9    LDH_D   0.000000
 Aeróbico meso-BDO     0.9   ALCD19   0.000000
 Aeróbico meso-BDO     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
          Condicao  Fracao        Reacao       Fluxo  Rank_abs
 Aeróbico meso-BDO     0.9      ATPS4rpp  118.615882         1
 Aeróbico meso-BDO     0.9        H2Otpp  -76.906070         2
 Aeróbico meso-BDO     0.9        H2Otex  -76.906070         3
 Aeróbico meso-BDO     0.9      EX_h2o_e   76.906070         4
 Aeróbico mes


=== Fração 0.3 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico meso-BDO | frac=0.30

  • Reações monitoradas:
            Condicao  Fracao   Reacao      Fluxo
 Anaeróbico meso-BDO     0.3     GAPD  36.588829
 Anaeróbico meso-BDO     0.3      MDH   0.114585
 Anaeróbico meso-BDO     0.3   13PPDH   0.000000
 Anaeróbico meso-BDO     0.3     DCTR   0.000000
 Anaeróbico meso-BDO     0.3  BTDD_RS  26.540482
 Anaeróbico meso-BDO     0.3  BTDD_SS   0.000000
 Anaeróbico meso-BDO     0.3   ALCD2x  -4.385478
 Anaeróbico meso-BDO     0.3    LDH_D   0.000000
 Anaeróbico meso-BDO     0.3   ALCD19   0.000000
 Anaeróbico meso-BDO     0.3    GDH_R   0.000000

  • Top20 (abs) desta solução:
            Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico meso-BDO     0.3           ENO  36.588829         1
 Anaeróbico meso-BDO     0.3          GAPD  36.588829         2
 Anaeróbico meso-BDO     0.3           PGK -36.588829         3
 Anaeróbico meso-BDO     0.3           PGM -36.5

Knockouts aplicados: GDH_R, DAC

=== Fração 0.7 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico meso-BDO | frac=0.70

  • Reações monitoradas:
            Condicao  Fracao   Reacao      Fluxo
 Anaeróbico meso-BDO     0.7     GAPD  36.040601
 Anaeróbico meso-BDO     0.7      MDH   0.267364
 Anaeróbico meso-BDO     0.7   13PPDH   0.000000
 Anaeróbico meso-BDO     0.7     DCTR   0.000000
 Anaeróbico meso-BDO     0.7  BTDD_RS  22.794043
 Anaeróbico meso-BDO     0.7  BTDD_SS   0.000000
 Anaeróbico meso-BDO     0.7   ALCD2x -11.213605
 Anaeróbico meso-BDO     0.7    LDH_D   0.000000
 Anaeróbico meso-BDO     0.7   ALCD19   0.000000
 Anaeróbico meso-BDO     0.7    GDH_R   0.000000

  • Top20 (abs) desta solução:
            Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico meso-BDO     0.7           PGK -36.040601         1
 Anaeróbico meso-BDO     0.7          GAPD  36.040601         2
 Anaeróbico meso-BDO     0.7           PGM -35.498476         3
 Anaeróbico meso

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,3.700000e+01
1,Aeróbico etanol,0.1,optimal,0.186515,3.456049e+01
2,Aeróbico etanol,0.2,optimal,0.373031,3.212099e+01
3,Aeróbico etanol,0.3,optimal,0.559546,2.825021e+01
4,Aeróbico etanol,0.4,optimal,0.746062,2.421447e+01
...,...,...,...,...,...
83,Anaeróbico meso-BDO,0.6,optimal,0.301936,1.239618e+01
84,Anaeróbico meso-BDO,0.7,optimal,0.352258,9.844615e+00
85,Anaeróbico meso-BDO,0.8,optimal,0.402581,6.563077e+00
86,Anaeróbico meso-BDO,0.9,optimal,0.452903,3.281538e+00


In [15]:
#Glicerol e etanol
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_etoh_e", 
                      condicao="2. Aeróbico etanol")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_etoh_e", 
                      condicao="2. Anaeróbico etanol")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: 2. Aeróbico etanol ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (2. Aeróbico etanol): 2.1341

Knockouts aplicados: nenhum

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico etanol | frac=0.00

  • Reações monitoradas:
           Condicao  Fracao   Reacao         Fluxo
 2. Aeróbico etanol     0.0     GAPD  3.324733e+01
 2. Aeróbico etanol     0.0      MDH -9.275477e-15
 2. Aeróbico etanol     0.0   13PPDH  0.000000e+00
 2. Aeróbico etanol     0.0     DCTR  0.000000e+00
 2. Aeróbico etanol     0.0  BTDD_RS  0.000000e+00
 2. Aeróbico etanol     0.0  BTDD_SS  0.000000e+00
 2. Aeróbico etanol     0.0   ALCD2x -4.249733e+01
 2. Aeróbico etanol     0.0    LDH_D  6.166667e+00
 2. Aeróbico etanol     0.0   ALCD19  0.000000e+00
 2. Aeróbico etanol     0.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
           Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Aeróbico etanol     0.0     ALCD2x -42.497333   


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico etanol | frac=0.50

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico etanol     0.5     GAPD  36.507793
 2. Aeróbico etanol     0.5      MDH   4.162321
 2. Aeróbico etanol     0.5   13PPDH   0.000000
 2. Aeróbico etanol     0.5     DCTR   0.000000
 2. Aeróbico etanol     0.5  BTDD_RS   0.000000
 2. Aeróbico etanol     0.5  BTDD_SS   0.000000
 2. Aeróbico etanol     0.5   ALCD2x -23.088596
 2. Aeróbico etanol     0.5    LDH_D   0.000000
 2. Aeróbico etanol     0.5   ALCD19   0.000000
 2. Aeróbico etanol     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
           Condicao  Fracao      Reacao      Fluxo  Rank_abs
 2. Aeróbico etanol     0.5    ATPS4rpp  66.437759         1
 2. Aeróbico etanol     0.5      H2Otex -52.654527         2
 2. Aeróbico etanol     0.5      H2Otpp -52.654527         3
 2. Aeróbico etanol     0.5    EX_h2o_e  52.654527         4
 2. Aeróbic


🔍 Monitorando fermentações alternativas:
  Etanol: ALCD2x=-9.2354, EX_etoh_e=9.2354
  L-Lactato: L_LACD2=0.0000, EX_lac__L_e=0.0000
  D-Lactato: LDH_D=0.0000, EX_lac__D_e=0.0000
  1,3-Propanodiol: 13PPDH=0.0000, EX_13ppd_e=0.0000
  BDO meso: BTDD_MESO=0.0000, EX_btd__RS_e=0.0000
  BDO S,S: BTDD_SS=0.0000, EX_btd__SS_e=0.0000
Knockouts aplicados: nenhum

=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico etanol | frac=0.90

  • Reações monitoradas:
           Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico etanol     0.9     GAPD  34.182828
 2. Aeróbico etanol     0.9      MDH  15.442154
 2. Aeróbico etanol     0.9   13PPDH   0.000000
 2. Aeróbico etanol     0.9     DCTR   0.000000
 2. Aeróbico etanol     0.9  BTDD_RS   0.000000
 2. Aeróbico etanol     0.9  BTDD_SS   0.000000
 2. Aeróbico etanol     0.9   ALCD2x  -4.617719
 2. Aeróbico etanol     0.9    LDH_D   0.000000
 2. Aeróbico etanol     0.9   ALCD19   0.000000
 2. Aeróbico etanol     0.9    GDH_R   0

Knockouts aplicados: DAC

=== Fração 0.3 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico etanol | frac=0.30

  • Reações monitoradas:
             Condicao  Fracao   Reacao      Fluxo
 2. Anaeróbico etanol     0.3     GAPD  39.026075
 2. Anaeróbico etanol     0.3      MDH   0.108106
 2. Anaeróbico etanol     0.3   13PPDH   0.000000
 2. Anaeróbico etanol     0.3     DCTR   0.000000
 2. Anaeróbico etanol     0.3  BTDD_RS  10.224404
 2. Anaeróbico etanol     0.3  BTDD_SS   0.000000
 2. Anaeróbico etanol     0.3   ALCD2x -40.634401
 2. Anaeróbico etanol     0.3    LDH_D   0.000000
 2. Anaeróbico etanol     0.3   ALCD19   0.000000
 2. Anaeróbico etanol     0.3    GDH_R   0.000000

  • Top20 (abs) desta solução:
             Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico etanol     0.3     ALCD2x -40.634401         1
 2. Anaeróbico etanol     0.3  EX_etoh_e  40.634401         2
 2. Anaeróbico etanol     0.3  ETOHt2rpp -40.634401         3
 2. Anaeróbico etan


=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico etanol | frac=0.90

  • Reações monitoradas:
             Condicao  Fracao   Reacao      Fluxo
 2. Anaeróbico etanol     0.9     GAPD  38.250225
 2. Anaeróbico etanol     0.9      MDH   0.324319
 2. Anaeróbico etanol     0.9   13PPDH   0.000000
 2. Anaeróbico etanol     0.9     DCTR   0.000000
 2. Anaeróbico etanol     0.9  BTDD_RS   2.106410
 2. Anaeróbico etanol     0.9  BTDD_SS   0.000000
 2. Anaeróbico etanol     0.9   ALCD2x -32.797335
 2. Anaeróbico etanol     0.9    LDH_D   0.000000
 2. Anaeróbico etanol     0.9   ALCD19   0.000000
 2. Anaeróbico etanol     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
             Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico etanol     0.9       GAPD  38.250225         1
 2. Anaeróbico etanol     0.9        PGK -38.250225         2
 2. Anaeróbico etanol     0.9        ENO  37.592612         3
 2. Anaeróbico etanol     0.9        PGM -37

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,37.000000
1,Aeróbico etanol,0.1,optimal,0.186515,34.560494
2,Aeróbico etanol,0.2,optimal,0.373031,32.120988
3,Aeróbico etanol,0.3,optimal,0.559546,28.250213
4,Aeróbico etanol,0.4,optimal,0.746062,24.214468
...,...,...,...,...,...
105,2. Anaeróbico etanol,0.6,optimal,0.284866,38.771468
106,2. Anaeróbico etanol,0.7,optimal,0.332343,37.390842
107,2. Anaeróbico etanol,0.8,optimal,0.379821,35.484010
108,2. Anaeróbico etanol,0.9,optimal,0.427298,32.797335


In [17]:
#Glicerol e R,R-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd_RR_e", 
                      condicao="2. Aeróbico R,R-BDO")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd_RR_e", 
                      condicao="2. Anaeróbico R,R-BDO")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: 2. Aeróbico R,R-BDO ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (2. Aeróbico R,R-BDO): 2.1341

Knockouts aplicados: nenhum

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico R,R-BDO | frac=0.00

  • Reações monitoradas:
            Condicao  Fracao   Reacao         Fluxo
 2. Aeróbico R,R-BDO     0.0     GAPD  3.941400e+01
 2. Aeróbico R,R-BDO     0.0      MDH -1.189039e-13
 2. Aeróbico R,R-BDO     0.0   13PPDH  0.000000e+00
 2. Aeróbico R,R-BDO     0.0     DCTR  0.000000e+00
 2. Aeróbico R,R-BDO     0.0  BTDD_RS  0.000000e+00
 2. Aeróbico R,R-BDO     0.0  BTDD_SS  0.000000e+00
 2. Aeróbico R,R-BDO     0.0   ALCD2x  0.000000e+00
 2. Aeróbico R,R-BDO     0.0    LDH_D  0.000000e+00
 2. Aeróbico R,R-BDO     0.0   ALCD19  0.000000e+00
 2. Aeróbico R,R-BDO     0.0    GDH_R  1.970700e+01

  • Top20 (abs) desta solução:
            Condicao  Fracao       Reacao    Fluxo  Rank_abs
 2. Aeróbico R,R-BDO     0.0         


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico R,R-BDO | frac=0.50

  • Reações monitoradas:
            Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico R,R-BDO     0.5     GAPD  36.507793
 2. Aeróbico R,R-BDO     0.5      MDH   1.013877
 2. Aeróbico R,R-BDO     0.5   13PPDH   0.000000
 2. Aeróbico R,R-BDO     0.5     DCTR   0.000000
 2. Aeróbico R,R-BDO     0.5  BTDD_RS   6.310780
 2. Aeróbico R,R-BDO     0.5  BTDD_SS   0.000000
 2. Aeróbico R,R-BDO     0.5   ALCD2x   0.000000
 2. Aeróbico R,R-BDO     0.5    LDH_D   0.000000
 2. Aeróbico R,R-BDO     0.5   ALCD19   0.000000
 2. Aeróbico R,R-BDO     0.5    GDH_R  12.331409

  • Top20 (abs) desta solução:
            Condicao  Fracao      Reacao      Fluxo  Rank_abs
 2. Aeróbico R,R-BDO     0.5    ATPS4rpp  68.011981         1
 2. Aeróbico R,R-BDO     0.5      H2Otpp -60.263268         2
 2. Aeróbico R,R-BDO     0.5    EX_h2o_e  60.263268         3
 2. Aeróbico R,R-BDO     0.5      H2Otex -60.263268     


=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico R,R-BDO | frac=0.90

  • Reações monitoradas:
            Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico R,R-BDO     0.9     GAPD  34.182828
 2. Aeróbico R,R-BDO     0.9      MDH  15.127310
 2. Aeróbico R,R-BDO     0.9   13PPDH   0.000000
 2. Aeróbico R,R-BDO     0.9     DCTR   0.000000
 2. Aeróbico R,R-BDO     0.9  BTDD_RS  14.176140
 2. Aeróbico R,R-BDO     0.9  BTDD_SS   0.000000
 2. Aeróbico R,R-BDO     0.9   ALCD2x   0.000000
 2. Aeróbico R,R-BDO     0.9    LDH_D   0.000000
 2. Aeróbico R,R-BDO     0.9   ALCD19   0.000000
 2. Aeróbico R,R-BDO     0.9    GDH_R   2.466282

  • Top20 (abs) desta solução:
            Condicao  Fracao      Reacao       Fluxo  Rank_abs
 2. Aeróbico R,R-BDO     0.9    ATPS4rpp  140.583457         1
 2. Aeróbico R,R-BDO     0.9      H2Otex  -97.731556         2
 2. Aeróbico R,R-BDO     0.9      H2Otpp  -97.731556         3
 2. Aeróbico R,R-BDO     0.9    EX_h2o_e   97.731556


=== Fração 0.3 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico R,R-BDO | frac=0.30

  • Reações monitoradas:
              Condicao  Fracao   Reacao      Fluxo
 2. Anaeróbico R,R-BDO     0.3     GAPD  39.026075
 2. Anaeróbico R,R-BDO     0.3      MDH   0.108106
 2. Anaeróbico R,R-BDO     0.3   13PPDH   0.000000
 2. Anaeróbico R,R-BDO     0.3     DCTR   0.000000
 2. Anaeróbico R,R-BDO     0.3  BTDD_RS  17.323449
 2. Anaeróbico R,R-BDO     0.3  BTDD_SS   0.000000
 2. Anaeróbico R,R-BDO     0.3   ALCD2x -12.018716
 2. Anaeróbico R,R-BDO     0.3    LDH_D   0.000000
 2. Anaeróbico R,R-BDO     0.3   ALCD19   0.000000
 2. Anaeróbico R,R-BDO     0.3    GDH_R  15.466803

  • Top20 (abs) desta solução:
              Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico R,R-BDO     0.3        PGK -39.026075         1
 2. Anaeróbico R,R-BDO     0.3        PGM -39.026075         2
 2. Anaeróbico R,R-BDO     0.3        ENO  39.026075         3
 2. Anaeróbico R,R-BDO     0


=== Fração 1.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico R,R-BDO | frac=1.00

  • Reações monitoradas:
              Condicao  Fracao   Reacao         Fluxo
 2. Anaeróbico R,R-BDO     1.0     GAPD  3.812092e+01
 2. Anaeróbico R,R-BDO     1.0      MDH  3.603549e-01
 2. Anaeróbico R,R-BDO     1.0   13PPDH  0.000000e+00
 2. Anaeróbico R,R-BDO     1.0     DCTR  0.000000e+00
 2. Anaeróbico R,R-BDO     1.0  BTDD_RS  2.779623e+00
 2. Anaeróbico R,R-BDO     1.0  BTDD_SS  0.000000e+00
 2. Anaeróbico R,R-BDO     1.0   ALCD2x -2.577067e+01
 2. Anaeróbico R,R-BDO     1.0    LDH_D  0.000000e+00
 2. Anaeróbico R,R-BDO     1.0   ALCD19  0.000000e+00
 2. Anaeróbico R,R-BDO     1.0    GDH_R -1.392751e-11

  • Top20 (abs) desta solução:
              Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico R,R-BDO     1.0       GAPD  38.120916         1
 2. Anaeróbico R,R-BDO     1.0        PGK -38.120916         2
 2. Anaeróbico R,R-BDO     1.0        PGM -37.390236      

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,3.700000e+01
1,Aeróbico etanol,0.1,optimal,0.186515,3.456049e+01
2,Aeróbico etanol,0.2,optimal,0.373031,3.212099e+01
3,Aeróbico etanol,0.3,optimal,0.559546,2.825021e+01
4,Aeróbico etanol,0.4,optimal,0.746062,2.421447e+01
...,...,...,...,...,...
149,"2. Anaeróbico R,R-BDO",0.6,optimal,0.284866,9.787060e+00
150,"2. Anaeróbico R,R-BDO",0.7,optimal,0.332343,7.893812e+00
151,"2. Anaeróbico R,R-BDO",0.8,optimal,0.379821,5.949477e+00
152,"2. Anaeróbico R,R-BDO",0.9,optimal,0.427298,3.096015e+00


In [16]:
#Glicerol e S,S-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__SS_e", 
                      condicao="2. Aeróbico S,S-BDO")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__SS_e", 
                      condicao="2. Anaeróbico S,S-BDO")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: 2. Aeróbico S,S-BDO ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (2. Aeróbico S,S-BDO): 2.1341

Knockouts aplicados: nenhum

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico S,S-BDO | frac=0.00

  • Reações monitoradas:
            Condicao  Fracao   Reacao         Fluxo
 2. Aeróbico S,S-BDO     0.0     GAPD  3.941400e+01
 2. Aeróbico S,S-BDO     0.0      MDH -1.779997e-13
 2. Aeróbico S,S-BDO     0.0   13PPDH  0.000000e+00
 2. Aeróbico S,S-BDO     0.0     DCTR  1.970700e+01
 2. Aeróbico S,S-BDO     0.0  BTDD_RS  0.000000e+00
 2. Aeróbico S,S-BDO     0.0  BTDD_SS -1.970700e+01
 2. Aeróbico S,S-BDO     0.0   ALCD2x  0.000000e+00
 2. Aeróbico S,S-BDO     0.0    LDH_D  0.000000e+00
 2. Aeróbico S,S-BDO     0.0   ALCD19  0.000000e+00
 2. Aeróbico S,S-BDO     0.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
            Condicao  Fracao        Reacao     Fluxo  Rank_abs
 2. Aeróbico S,S-BDO     0.0       


=== Fração 0.6 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico S,S-BDO | frac=0.60

  • Reações monitoradas:
            Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico S,S-BDO     0.6     GAPD  35.926552
 2. Aeróbico S,S-BDO     0.6      MDH   7.716566
 2. Aeróbico S,S-BDO     0.6   13PPDH   0.000000
 2. Aeróbico S,S-BDO     0.6     DCTR   8.949806
 2. Aeróbico S,S-BDO     0.6  BTDD_RS   0.000000
 2. Aeróbico S,S-BDO     0.6  BTDD_SS  -8.949806
 2. Aeróbico S,S-BDO     0.6   ALCD2x   0.000000
 2. Aeróbico S,S-BDO     0.6    LDH_D   0.000000
 2. Aeróbico S,S-BDO     0.6   ALCD19   0.000000
 2. Aeróbico S,S-BDO     0.6    GDH_R   0.000000

  • Top20 (abs) desta solução:
            Condicao  Fracao      Reacao      Fluxo  Rank_abs
 2. Aeróbico S,S-BDO     0.6    ATPS4rpp  84.324208         1
 2. Aeróbico S,S-BDO     0.6      H2Otpp -65.257140         2
 2. Aeróbico S,S-BDO     0.6      H2Otex -65.257140         3
 2. Aeróbico S,S-BDO     0.6    EX_h2o_e  65.257140     

Valor da função objetivo: 2.2375
Knockouts aplicados: nenhum

=== Fração 1.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico S,S-BDO | frac=1.00

  • Reações monitoradas:
            Condicao  Fracao   Reacao         Fluxo
 2. Aeróbico S,S-BDO     1.0     GAPD  3.360159e+01
 2. Aeróbico S,S-BDO     1.0      MDH  1.790941e+01
 2. Aeróbico S,S-BDO     1.0   13PPDH  0.000000e+00
 2. Aeróbico S,S-BDO     1.0     DCTR  4.030560e-13
 2. Aeróbico S,S-BDO     1.0  BTDD_RS  0.000000e+00
 2. Aeróbico S,S-BDO     1.0  BTDD_SS -4.030560e-13
 2. Aeróbico S,S-BDO     1.0   ALCD2x  0.000000e+00
 2. Aeróbico S,S-BDO     1.0    LDH_D  0.000000e+00
 2. Aeróbico S,S-BDO     1.0   ALCD19  0.000000e+00
 2. Aeróbico S,S-BDO     1.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
            Condicao  Fracao      Reacao       Fluxo  Rank_abs
 2. Aeróbico S,S-BDO     1.0    ATPS4rpp  158.726326         1
 2. Aeróbico S,S-BDO     1.0      H2Otpp -107.098628         2
 2. Aeróbico S,S-BDO 


📊 Monitoramento — 2. Anaeróbico S,S-BDO | frac=0.40

  • Reações monitoradas:
              Condicao  Fracao   Reacao      Fluxo
 2. Anaeróbico S,S-BDO     0.4     GAPD  38.120916
 2. Anaeróbico S,S-BDO     0.4      MDH   0.360355
 2. Anaeróbico S,S-BDO     0.4   13PPDH   0.000000
 2. Anaeróbico S,S-BDO     0.4     DCTR   0.000000
 2. Anaeróbico S,S-BDO     0.4  BTDD_RS   5.960527
 2. Anaeróbico S,S-BDO     0.4  BTDD_SS   0.000000
 2. Anaeróbico S,S-BDO     0.4   ALCD2x -25.770668
 2. Anaeróbico S,S-BDO     0.4    LDH_D   0.000000
 2. Anaeróbico S,S-BDO     0.4   ALCD19   0.000000
 2. Anaeróbico S,S-BDO     0.4    GDH_R   0.000000

  • Top20 (abs) desta solução:
              Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico S,S-BDO     0.4        PGK -38.120916         1
 2. Anaeróbico S,S-BDO     0.4       GAPD  38.120916         2
 2. Anaeróbico S,S-BDO     0.4        PGM -37.390236         3
 2. Anaeróbico S,S-BDO     0.4        ENO  37.390236         4
 2. Anaeróbic

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,37.000000
1,Aeróbico etanol,0.1,optimal,0.186515,34.560494
2,Aeróbico etanol,0.2,optimal,0.373031,32.120988
3,Aeróbico etanol,0.3,optimal,0.559546,28.250213
4,Aeróbico etanol,0.4,optimal,0.746062,24.214468
...,...,...,...,...,...
127,"2. Anaeróbico S,S-BDO",0.6,optimal,0.474776,0.000000
128,"2. Anaeróbico S,S-BDO",0.7,optimal,0.474776,0.000000
129,"2. Anaeróbico S,S-BDO",0.8,optimal,0.474776,0.000000
130,"2. Anaeróbico S,S-BDO",0.9,optimal,0.474776,0.000000


In [18]:
#Glicerol e meso-BDO 
# Aeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__RS_e", 
                      condicao="2. Aeróbico meso-BDO ")

# Anaeróbico
simular_fba_iterativo(model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo="EX_btd__RS_e", 
                      condicao="2. Anaeróbico meso-BDO ")

# Salvar tudo no final
#salvar_resultados("resultados_iterativos_fba.xlsx")



=== Iniciando simulação: 2. Aeróbico meso-BDO  ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (2. Aeróbico meso-BDO ): 2.1341

Knockouts aplicados: nenhum

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico meso-BDO  | frac=0.00

  • Reações monitoradas:
              Condicao  Fracao   Reacao         Fluxo
 2. Aeróbico meso-BDO      0.0     GAPD  3.941400e+01
 2. Aeróbico meso-BDO      0.0      MDH  4.275399e-14
 2. Aeróbico meso-BDO      0.0   13PPDH  0.000000e+00
 2. Aeróbico meso-BDO      0.0     DCTR  0.000000e+00
 2. Aeróbico meso-BDO      0.0  BTDD_RS  1.970700e+01
 2. Aeróbico meso-BDO      0.0  BTDD_SS  0.000000e+00
 2. Aeróbico meso-BDO      0.0   ALCD2x  0.000000e+00
 2. Aeróbico meso-BDO      0.0    LDH_D  0.000000e+00
 2. Aeróbico meso-BDO      0.0   ALCD19  0.000000e+00
 2. Aeróbico meso-BDO      0.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
              Condicao  Fracao      Reacao    Fluxo  Rank_abs
 2. Aeró


📊 Monitoramento — 2. Aeróbico meso-BDO  | frac=0.50

  • Reações monitoradas:
              Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico meso-BDO      0.5     GAPD  36.507793
 2. Aeróbico meso-BDO      0.5      MDH   1.013877
 2. Aeróbico meso-BDO      0.5   13PPDH   0.000000
 2. Aeróbico meso-BDO      0.5     DCTR   0.000000
 2. Aeróbico meso-BDO      0.5  BTDD_RS  12.331409
 2. Aeróbico meso-BDO      0.5  BTDD_SS  -6.310780
 2. Aeróbico meso-BDO      0.5   ALCD2x   0.000000
 2. Aeróbico meso-BDO      0.5    LDH_D   0.000000
 2. Aeróbico meso-BDO      0.5   ALCD19   0.000000
 2. Aeróbico meso-BDO      0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
              Condicao  Fracao      Reacao      Fluxo  Rank_abs
 2. Aeróbico meso-BDO      0.5    ATPS4rpp  68.011981         1
 2. Aeróbico meso-BDO      0.5      H2Otpp -60.263268         2
 2. Aeróbico meso-BDO      0.5      H2Otex -60.263268         3
 2. Aeróbico meso-BDO      0.5    EX_h2o_e  60.263268         4
 2. Aeró


=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Aeróbico meso-BDO  | frac=0.90

  • Reações monitoradas:
              Condicao  Fracao   Reacao      Fluxo
 2. Aeróbico meso-BDO      0.9     GAPD  34.182828
 2. Aeróbico meso-BDO      0.9      MDH  15.127310
 2. Aeróbico meso-BDO      0.9   13PPDH   0.000000
 2. Aeróbico meso-BDO      0.9     DCTR   0.000000
 2. Aeróbico meso-BDO      0.9  BTDD_RS   2.466282
 2. Aeróbico meso-BDO      0.9  BTDD_SS -14.176140
 2. Aeróbico meso-BDO      0.9   ALCD2x   0.000000
 2. Aeróbico meso-BDO      0.9    LDH_D   0.000000
 2. Aeróbico meso-BDO      0.9   ALCD19   0.000000
 2. Aeróbico meso-BDO      0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
              Condicao  Fracao      Reacao       Fluxo  Rank_abs
 2. Aeróbico meso-BDO      0.9    ATPS4rpp  140.583457         1
 2. Aeróbico meso-BDO      0.9      H2Otpp  -97.731556         2
 2. Aeróbico meso-BDO      0.9      H2Otex  -97.731556         3
 2. Aeróbico meso-BD


=== Fração 0.3 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico meso-BDO  | frac=0.30

  • Reações monitoradas:
                Condicao  Fracao   Reacao      Fluxo
 2. Anaeróbico meso-BDO      0.3     GAPD  39.026075
 2. Anaeróbico meso-BDO      0.3      MDH   0.108106
 2. Anaeróbico meso-BDO      0.3   13PPDH   0.000000
 2. Anaeróbico meso-BDO      0.3     DCTR   0.000000
 2. Anaeróbico meso-BDO      0.3  BTDD_RS  15.466803
 2. Anaeróbico meso-BDO      0.3  BTDD_SS -29.056808
 2. Anaeróbico meso-BDO      0.3   ALCD2x -12.018716
 2. Anaeróbico meso-BDO      0.3    LDH_D   0.000000
 2. Anaeróbico meso-BDO      0.3   ALCD19   0.000000
 2. Anaeróbico meso-BDO      0.3    GDH_R   0.000000

  • Top20 (abs) desta solução:
                Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico meso-BDO      0.3        PGM -39.026075         1
 2. Anaeróbico meso-BDO      0.3        PGK -39.026075         2
 2. Anaeróbico meso-BDO      0.3       GAPD  39.026075       


=== Fração 1.0 da biomassa ===
Status: optimal

📊 Monitoramento — 2. Anaeróbico meso-BDO  | frac=1.00

  • Reações monitoradas:
                Condicao  Fracao   Reacao         Fluxo
 2. Anaeróbico meso-BDO      1.0     GAPD  3.812092e+01
 2. Anaeróbico meso-BDO      1.0      MDH  3.603549e-01
 2. Anaeróbico meso-BDO      1.0   13PPDH  0.000000e+00
 2. Anaeróbico meso-BDO      1.0     DCTR  0.000000e+00
 2. Anaeróbico meso-BDO      1.0  BTDD_RS -2.340194e-11
 2. Anaeróbico meso-BDO      1.0  BTDD_SS -5.996705e+00
 2. Anaeróbico meso-BDO      1.0   ALCD2x -2.577067e+01
 2. Anaeróbico meso-BDO      1.0    LDH_D  0.000000e+00
 2. Anaeróbico meso-BDO      1.0   ALCD19  0.000000e+00
 2. Anaeróbico meso-BDO      1.0    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
                Condicao  Fracao     Reacao      Fluxo  Rank_abs
 2. Anaeróbico meso-BDO      1.0        PGK -38.120916         1
 2. Anaeróbico meso-BDO      1.0       GAPD  38.120916         2
 2. Anaeróbico meso-BDO     

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,3.700000e+01
1,Aeróbico etanol,0.1,optimal,0.186515,3.456049e+01
2,Aeróbico etanol,0.2,optimal,0.373031,3.212099e+01
3,Aeróbico etanol,0.3,optimal,0.559546,2.825021e+01
4,Aeróbico etanol,0.4,optimal,0.746062,2.421447e+01
...,...,...,...,...,...
171,2. Anaeróbico meso-BDO,0.6,optimal,0.284866,9.787060e+00
172,2. Anaeróbico meso-BDO,0.7,optimal,0.332343,7.893812e+00
173,2. Anaeróbico meso-BDO,0.8,optimal,0.379821,5.949477e+00
174,2. Anaeróbico meso-BDO,0.9,optimal,0.427298,3.096015e+00


In [19]:
# Exemplo de lista de knockouts
knockouts_exemplo = ["LDH_D"]

# Rodar simulação com knockouts
simular_fba_iterativo(
    model,
    fonte_carbono="EX_glc__D_e",
    qtd_carbono=18.5,
    qtd_oxigenio=0,
    objetivo_biomassa="BIOMASS_",
    objetivos_alvo= "EX_etoh_e",
    condicao="Anaeróbico_com_knockout_ALCD2x_LDH_D",
    passos=10,
    nocautes=knockouts_exemplo
)



=== Iniciando simulação: Anaeróbico_com_knockout_ALCD2x_LDH_D ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Anaeróbico_com_knockout_ALCD2x_LDH_D): 0.5032

Knockouts aplicados: GDH_R, DAC, LDH_D

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico_com_knockout_ALCD2x_LDH_D | frac=0.00

  • Reações monitoradas:
                             Condicao  Fracao   Reacao         Fluxo
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0     GAPD  3.700000e+01
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0      MDH  1.865360e-13
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0   13PPDH  0.000000e+00
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0     DCTR  0.000000e+00
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0  BTDD_RS  0.000000e+00
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0  BTDD_SS -7.013696e-13
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0   ALCD2x -3.700000e+01
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.0    LDH_D  0.000000e+00
 Anaeróbico_com_kno


=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico_com_knockout_ALCD2x_LDH_D | frac=0.50

  • Reações monitoradas:
                             Condicao  Fracao   Reacao      Fluxo
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5     GAPD  36.314715
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5      MDH   0.638753
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5   13PPDH   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5     DCTR   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5  BTDD_RS   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5  BTDD_SS  -5.087011
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5   ALCD2x -33.709057
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5    LDH_D   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5   ALCD19   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
                             Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico_com_kno

Knockouts aplicados: GDH_R, DAC, LDH_D

=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico_com_knockout_ALCD2x_LDH_D | frac=0.90

  • Reações monitoradas:
                             Condicao  Fracao   Reacao      Fluxo
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9     GAPD  35.766487
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9      MDH   0.343754
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9   13PPDH   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9     DCTR   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9  BTDD_RS   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9  BTDD_SS  -5.720441
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9   ALCD2x -22.570460
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9    LDH_D   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9   ALCD19   0.000000
 Anaeróbico_com_knockout_ALCD2x_LDH_D     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
                             Condicao  Fracao        Reacao  

,Condicao,Fracao,Status,Biomassa,Fluxo_alvo
0,Aeróbico etanol,0.0,optimal,0.000000,37.000000
1,Aeróbico etanol,0.1,optimal,0.186515,34.560494
2,Aeróbico etanol,0.2,optimal,0.373031,32.120988
3,Aeróbico etanol,0.3,optimal,0.559546,28.250213
4,Aeróbico etanol,0.4,optimal,0.746062,24.214468
...,...,...,...,...,...
182,Anaeróbico_com_knockout_ALCD2x_LDH_D,0.6,optimal,0.301936,33.050869
183,Anaeróbico_com_knockout_ALCD2x_LDH_D,0.7,optimal,0.352258,32.392680
184,Anaeróbico_com_knockout_ALCD2x_LDH_D,0.8,optimal,0.402581,30.018186
185,Anaeróbico_com_knockout_ALCD2x_LDH_D,0.9,optimal,0.452903,22.570460


In [20]:
#Glicose e etanol 
# Aeróbico
#monitorar outras fermentações

objetivos = ["EX_etoh_e", "EX_lac__L_e", "EX_lac__D_e", "EX_13ppd_e", "EX_btd__RS_e", "EX_btd__SS_e"]

resultado_aerobico= simular_fba_iterativo(
                      model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo=objetivos, 
                      condicao="Aeróbico etanol - nocautes_1")

# Anaeróbico
resultado_anaerobico = simular_fba_iterativo(
                      model, 
                      fonte_carbono="EX_glc__D_e", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo=objetivos, 
                      condicao="Anaeróbico etanol- nocautes_1")



=== Iniciando simulação: Aeróbico etanol - nocautes_1 ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico etanol - nocautes_1): 1.8652

Knockouts aplicados: GDH_R
Função objetivo composta: EX_etoh_e, EX_lac__L_e, EX_lac__D_e, EX_13ppd_e, EX_btd__RS_e, EX_btd__SS_e

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol - nocautes_1 | frac=0.00

  • Reações monitoradas:
                     Condicao  Fracao   Reacao         Fluxo
 Aeróbico etanol - nocautes_1     0.0     GAPD  3.700000e+01
 Aeróbico etanol - nocautes_1     0.0      MDH -9.991640e-14
 Aeróbico etanol - nocautes_1     0.0   13PPDH  0.000000e+00
 Aeróbico etanol - nocautes_1     0.0     DCTR  0.000000e+00
 Aeróbico etanol - nocautes_1     0.0  BTDD_RS  0.000000e+00
 Aeróbico etanol - nocautes_1     0.0  BTDD_SS  0.000000e+00
 Aeróbico etanol - nocautes_1     0.0   ALCD2x  0.000000e+00
 Aeróbico etanol - nocautes_1     0.0    LDH_D -3.700000e+01
 Aeróbico etanol - nocaut


=== Fração 0.4 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol - nocautes_1 | frac=0.40

  • Reações monitoradas:
                     Condicao  Fracao   Reacao      Fluxo
 Aeróbico etanol - nocautes_1     0.4     GAPD  34.968052
 Aeróbico etanol - nocautes_1     0.4      MDH   6.016694
 Aeróbico etanol - nocautes_1     0.4   13PPDH   0.000000
 Aeróbico etanol - nocautes_1     0.4     DCTR  11.732784
 Aeróbico etanol - nocautes_1     0.4  BTDD_RS   2.906052
 Aeróbico etanol - nocautes_1     0.4  BTDD_SS -11.732784
 Aeróbico etanol - nocautes_1     0.4   ALCD2x   0.000000
 Aeróbico etanol - nocautes_1     0.4    LDH_D   0.000000
 Aeróbico etanol - nocautes_1     0.4   ALCD19   0.000000
 Aeróbico etanol - nocautes_1     0.4    GDH_R   0.000000

  • Top20 (abs) desta solução:
                     Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Aeróbico etanol - nocautes_1     0.4      ATPS4rpp  36.934444         1
 Aeróbico etanol - nocautes_1     0.4          GAP


=== Fração 0.7 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol - nocautes_1 | frac=0.70

  • Reações monitoradas:
                     Condicao  Fracao   Reacao      Fluxo
 Aeróbico etanol - nocautes_1     0.7     GAPD  31.920129
 Aeróbico etanol - nocautes_1     0.7      MDH  18.205653
 Aeróbico etanol - nocautes_1     0.7   13PPDH   0.000000
 Aeróbico etanol - nocautes_1     0.7     DCTR   0.000000
 Aeróbico etanol - nocautes_1     0.7  BTDD_RS   4.101213
 Aeróbico etanol - nocautes_1     0.7  BTDD_SS   0.000000
 Aeróbico etanol - nocautes_1     0.7   ALCD2x   0.000000
 Aeróbico etanol - nocautes_1     0.7    LDH_D   0.000000
 Aeróbico etanol - nocautes_1     0.7   ALCD19   0.000000
 Aeróbico etanol - nocautes_1     0.7    GDH_R   0.000000

  • Top20 (abs) desta solução:
                     Condicao  Fracao       Reacao       Fluxo  Rank_abs
 Aeróbico etanol - nocautes_1     0.7     ATPS4rpp  134.472192         1
 Aeróbico etanol - nocautes_1     0.7       H2Otpp

=== Iniciando simulação: Anaeróbico etanol- nocautes_1 ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Anaeróbico etanol- nocautes_1): 0.5032

Knockouts aplicados: GDH_R, DAC
Função objetivo composta: EX_etoh_e, EX_lac__L_e, EX_lac__D_e, EX_13ppd_e, EX_btd__RS_e, EX_btd__SS_e

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico etanol- nocautes_1 | frac=0.00

  • Reações monitoradas:
                      Condicao  Fracao   Reacao         Fluxo
 Anaeróbico etanol- nocautes_1     0.0     GAPD  3.700000e+01
 Anaeróbico etanol- nocautes_1     0.0      MDH -7.473242e-14
 Anaeróbico etanol- nocautes_1     0.0   13PPDH  0.000000e+00
 Anaeróbico etanol- nocautes_1     0.0     DCTR  0.000000e+00
 Anaeróbico etanol- nocautes_1     0.0  BTDD_RS  8.656761e-14
 Anaeróbico etanol- nocautes_1     0.0  BTDD_SS  0.000000e+00
 Anaeróbico etanol- nocautes_1     0.0   ALCD2x -3.700000e+01
 Anaeróbico etanol- nocautes_1     0.0    LDH_D  0.000000e+00
 Anaeróbi

Knockouts aplicados: GDH_R, DAC, EX_13ppd_e, EX_etoh_e, EX_btd__RS_e, EX_lac__D_e
Função objetivo composta: EX_etoh_e, EX_lac__L_e, EX_lac__D_e, EX_13ppd_e, EX_btd__RS_e, EX_btd__SS_e

=== Fração 0.5 da biomassa ===
Status: optimal

📊 Monitoramento — Anaeróbico etanol- nocautes_1 | frac=0.50

  • Reações monitoradas:
                      Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol- nocautes_1     0.5     GAPD  35.870800
 Anaeróbico etanol- nocautes_1     0.5      MDH   0.068942
 Anaeróbico etanol- nocautes_1     0.5   13PPDH   0.000000
 Anaeróbico etanol- nocautes_1     0.5     DCTR   0.000000
 Anaeróbico etanol- nocautes_1     0.5  BTDD_RS  38.482372
 Anaeróbico etanol- nocautes_1     0.5  BTDD_SS   0.000000
 Anaeróbico etanol- nocautes_1     0.5   ALCD2x   0.000000
 Anaeróbico etanol- nocautes_1     0.5    LDH_D   0.000000
 Anaeróbico etanol- nocautes_1     0.5   ALCD19   0.000000
 Anaeróbico etanol- nocautes_1     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:


/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)



=== Fração 0.7 da biomassa ===
Status: infeasible
⚠️ Solução inviável nesta fração.


📊 Monitoramento — Anaeróbico etanol- nocautes_1 | frac=0.70

  • Reações monitoradas:
                      Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol- nocautes_1     0.7     GAPD  36.040601
 Anaeróbico etanol- nocautes_1     0.7      MDH   0.096519
 Anaeróbico etanol- nocautes_1     0.7   13PPDH   0.000000
 Anaeróbico etanol- nocautes_1     0.7     DCTR   0.000000
 Anaeróbico etanol- nocautes_1     0.7  BTDD_RS  32.975410
 Anaeróbico etanol- nocautes_1     0.7  BTDD_SS   0.000000
 Anaeróbico etanol- nocautes_1     0.7   ALCD2x   0.000000
 Anaeróbico etanol- nocautes_1     0.7    LDH_D   0.000000
 Anaeróbico etanol- nocautes_1     0.7   ALCD19   0.000000
 Anaeróbico etanol- nocautes_1     0.7    GDH_R   0.000000

  • Top20 (abs) desta solução:
                      Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico etanol- nocautes_1     0.7           PGK -36.040601         

/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)



=== Fração 0.9 da biomassa ===
Status: infeasible
⚠️ Solução inviável nesta fração.


📊 Monitoramento — Anaeróbico etanol- nocautes_1 | frac=0.90

  • Reações monitoradas:
                      Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol- nocautes_1     0.9     GAPD  35.766487
 Anaeróbico etanol- nocautes_1     0.9      MDH   0.124096
 Anaeróbico etanol- nocautes_1     0.9   13PPDH   0.000000
 Anaeróbico etanol- nocautes_1     0.9     DCTR   0.000000
 Anaeróbico etanol- nocautes_1     0.9  BTDD_RS  23.196956
 Anaeróbico etanol- nocautes_1     0.9  BTDD_SS   0.000000
 Anaeróbico etanol- nocautes_1     0.9   ALCD2x   0.000000
 Anaeróbico etanol- nocautes_1     0.9    LDH_D   0.000000
 Anaeróbico etanol- nocautes_1     0.9   ALCD19   0.000000
 Anaeróbico etanol- nocautes_1     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
                      Condicao  Fracao        Reacao      Fluxo  Rank_abs
 Anaeróbico etanol- nocautes_1     0.9           PGK -35.766487         

In [21]:
#Glicerol e etanol 
# Aeróbico
#monitorar outras fermentações

objetivos = ["EX_etoh_e", "EX_lac__L_e", "EX_lac__D_e", "EX_13ppd_e", "EX_btd__RS_e", "EX_btd__SS_e"]

resultado_aerobico = simular_fba_iterativo(
                      model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=1000, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo=objetivos, 
                      condicao="Aeróbico etanol - nocautes_2")

# Anaeróbico
resultado_anaerobico = simular_fba_iterativo(
                      model, 
                      fonte_carbono="glicerol", 
                      qtd_carbono=18.5, 
                      qtd_oxigenio=0, 
                      objetivo_biomassa="BIOMASS_", 
                      objetivos_alvo=objetivos, 
                      condicao="Anaeróbico etanol- nocautes_2")




=== Iniciando simulação: Aeróbico etanol - nocautes_2 ===
Ajustando meio e otimizando biomassa máxima...

Biomassa máxima (Aeróbico etanol - nocautes_2): 2.1341

Knockouts aplicados: nenhum
Função objetivo composta: EX_etoh_e, EX_lac__L_e, EX_lac__D_e, EX_13ppd_e, EX_btd__RS_e, EX_btd__SS_e

=== Fração 0.0 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol - nocautes_2 | frac=0.00

  • Reações monitoradas:
                     Condicao  Fracao   Reacao         Fluxo
 Aeróbico etanol - nocautes_2     0.0     GAPD  3.941400e+01
 Aeróbico etanol - nocautes_2     0.0      MDH  2.433539e-13
 Aeróbico etanol - nocautes_2     0.0   13PPDH  0.000000e+00
 Aeróbico etanol - nocautes_2     0.0     DCTR  3.813790e-15
 Aeróbico etanol - nocautes_2     0.0  BTDD_RS  1.233333e+01
 Aeróbico etanol - nocautes_2     0.0  BTDD_SS -3.813790e-15
 Aeróbico etanol - nocautes_2     0.0   ALCD2x -4.249733e+01
 Aeróbico etanol - nocautes_2     0.0    LDH_D  4.485652e-15
 Aeróbico etanol - nocau


📊 Monitoramento — Aeróbico etanol - nocautes_2 | frac=0.50

  • Reações monitoradas:
                     Condicao  Fracao   Reacao      Fluxo
 Aeróbico etanol - nocautes_2     0.5     GAPD  36.507793
 Aeróbico etanol - nocautes_2     0.5      MDH   0.000000
 Aeróbico etanol - nocautes_2     0.5   13PPDH   0.000000
 Aeróbico etanol - nocautes_2     0.5     DCTR   0.000000
 Aeróbico etanol - nocautes_2     0.5  BTDD_RS  13.010716
 Aeróbico etanol - nocautes_2     0.5  BTDD_SS   0.000000
 Aeróbico etanol - nocautes_2     0.5   ALCD2x   0.000000
 Aeróbico etanol - nocautes_2     0.5    LDH_D   0.000000
 Aeróbico etanol - nocautes_2     0.5   ALCD19   0.000000
 Aeróbico etanol - nocautes_2     0.5    GDH_R   0.000000

  • Top20 (abs) desta solução:
                     Condicao  Fracao    Reacao      Fluxo  Rank_abs
 Aeróbico etanol - nocautes_2     0.5    H2Otpp -74.513592         1
 Aeróbico etanol - nocautes_2     0.5    H2Otex -74.513592         2
 Aeróbico etanol - nocautes_2     0.5

Função objetivo composta: EX_etoh_e, EX_lac__L_e, EX_lac__D_e, EX_13ppd_e, EX_btd__RS_e, EX_btd__SS_e

=== Fração 0.9 da biomassa ===
Status: optimal

📊 Monitoramento — Aeróbico etanol - nocautes_2 | frac=0.90

  • Reações monitoradas:
                     Condicao  Fracao   Reacao      Fluxo
 Aeróbico etanol - nocautes_2     0.9     GAPD  34.182828
 Aeróbico etanol - nocautes_2     0.9      MDH  12.372102
 Aeróbico etanol - nocautes_2     0.9   13PPDH   0.000000
 Aeróbico etanol - nocautes_2     0.9     DCTR   0.000000
 Aeróbico etanol - nocautes_2     0.9  BTDD_RS  12.146148
 Aeróbico etanol - nocautes_2     0.9  BTDD_SS   0.000000
 Aeróbico etanol - nocautes_2     0.9   ALCD2x   0.000000
 Aeróbico etanol - nocautes_2     0.9    LDH_D   0.000000
 Aeróbico etanol - nocautes_2     0.9   ALCD19   0.000000
 Aeróbico etanol - nocautes_2     0.9    GDH_R   0.000000

  • Top20 (abs) desta solução:
                     Condicao  Fracao    Reacao       Fluxo  Rank_abs
 Aeróbico etanol - nocau


📊 Monitoramento — Anaeróbico etanol- nocautes_2 | frac=0.20

  • Reações monitoradas:
                      Condicao  Fracao   Reacao         Fluxo
 Anaeróbico etanol- nocautes_2     0.2     GAPD  3.915538e+01
 Anaeróbico etanol- nocautes_2     0.2      MDH  0.000000e+00
 Anaeróbico etanol- nocautes_2     0.2   13PPDH  3.574663e-15
 Anaeróbico etanol- nocautes_2     0.2     DCTR  0.000000e+00
 Anaeróbico etanol- nocautes_2     0.2  BTDD_RS  4.433350e+01
 Anaeróbico etanol- nocautes_2     0.2  BTDD_SS  0.000000e+00
 Anaeróbico etanol- nocautes_2     0.2   ALCD2x  0.000000e+00
 Anaeróbico etanol- nocautes_2     0.2    LDH_D -1.274682e-14
 Anaeróbico etanol- nocautes_2     0.2   ALCD19  0.000000e+00
 Anaeróbico etanol- nocautes_2     0.2    GDH_R  0.000000e+00

  • Top20 (abs) desta solução:
                      Condicao  Fracao     Reacao      Fluxo  Rank_abs
 Anaeróbico etanol- nocautes_2     0.2    BTDD_RS  44.333500         1
 Anaeróbico etanol- nocautes_2     0.2   BTDDa_RS -39.321

/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


...

/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)



=== Fração 0.6 da biomassa ===
Status: infeasible
⚠️ Solução inviável nesta fração.


📊 Monitoramento — Anaeróbico etanol- nocautes_2 | frac=0.60

  • Reações monitoradas:
                      Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol- nocautes_2     0.6     GAPD  39.012986
 Anaeróbico etanol- nocautes_2     0.6      MDH   0.000000
 Anaeróbico etanol- nocautes_2     0.6   13PPDH   0.000000
 Anaeróbico etanol- nocautes_2     0.6     DCTR   0.000000
 Anaeróbico etanol- nocautes_2     0.6  BTDD_RS  47.423134
 Anaeróbico etanol- nocautes_2     0.6  BTDD_SS   0.000000
 Anaeróbico etanol- nocautes_2     0.6   ALCD2x   0.000000
 Anaeróbico etanol- nocautes_2     0.6    LDH_D   0.000000
 Anaeróbico etanol- nocautes_2     0.6   ALCD19   0.000000
 Anaeróbico etanol- nocautes_2     0.6    GDH_R   0.000000

  • Top20 (abs) desta solução:
                      Condicao  Fracao     Reacao      Fluxo  Rank_abs
 Anaeróbico etanol- nocautes_2     0.6    BTDD_RS  47.423134         1
 Ana

/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)



=== Fração 0.8 da biomassa ===
Status: infeasible
⚠️ Solução inviável nesta fração.


📊 Monitoramento — Anaeróbico etanol- nocautes_2 | frac=0.80

  • Reações monitoradas:
                      Condicao  Fracao   Reacao      Fluxo
 Anaeróbico etanol- nocautes_2     0.8     GAPD  39.012986
 Anaeróbico etanol- nocautes_2     0.8      MDH   0.000000
 Anaeróbico etanol- nocautes_2     0.8   13PPDH   0.000000
 Anaeróbico etanol- nocautes_2     0.8     DCTR   0.000000
 Anaeróbico etanol- nocautes_2     0.8  BTDD_RS  47.423134
 Anaeróbico etanol- nocautes_2     0.8  BTDD_SS   0.000000
 Anaeróbico etanol- nocautes_2     0.8   ALCD2x   0.000000
 Anaeróbico etanol- nocautes_2     0.8    LDH_D   0.000000
 Anaeróbico etanol- nocautes_2     0.8   ALCD19   0.000000
 Anaeróbico etanol- nocautes_2     0.8    GDH_R   0.000000

  • Top20 (abs) desta solução:
                      Condicao  Fracao     Reacao      Fluxo  Rank_abs
 Anaeróbico etanol- nocautes_2     0.8   BTDDa_RS -47.423134         1
 Ana

/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [22]:
salvar_resultados("resumo_simulacoes.xlsx")


Resultados consolidados salvos em 'resumo_simulacoes.xlsx'
